## **Collaborative filtering using matrix factorization**


In [21]:
import numpy as np
#collection of rating data
ratings = np.array(
    [
        [3.0, 3.0, 2.0, 3.0, 3.0, 3.0],
        [4.0, 1.0, 1.0, 4.0, 5.0, 3.0],
        [1.0, 2.0, 2.0, 1.0, 1.0, 2.0],
        [3.0, 2.0, 1.0, 3.0, 4.0, 3.0],
        [1.0, 5.0, 3.0, 1.0, 1.0, 3.0],
        [2.0, 5.0, 3.0, 2.0, 3.0, 4.0],
        [2.0, 2.0, 1.0, 2.0, 2.0, 2.0],
        [1.0, 4.0, 3.0, 1.0, 2.0, 3.0],
    ]
)

In [22]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
K = 2 #latent variable
R = ratings

In [23]:
N = len(ratings)
M = len(ratings[0])
P = np.random.rand(N, K)
Q = np.random.rand(M, K)
ratings = tf.placeholder(tf.float32, name="ratings")
P_matrix = tf.Variable(P, dtype=tf.float32)
Q_matrix = tf.Variable(Q, dtype=tf.float32)
P_times_Q = tf.matmul(P_matrix, Q_matrix, transpose_b=True)
squared_error = tf.square(P_times_Q - ratings)
loss = tf.reduce_sum(squared_error)

In [24]:
# gradient descent optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [25]:
#Train the model to calculate the P and Q matrices.
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(5000):
    sess.run(train, {ratings: R})

In [26]:
final_P_times_Q = np.around(sess.run(P_times_Q), 3)
print(final_P_times_Q)
print()
final_P_matrix = np.around(sess.run(P_matrix), 3)
print(final_P_matrix)
print()
final_Q_matrix = np.transpose(np.around(sess.run(Q_matrix), 3))
print(final_Q_matrix)
print()

[[2.721 2.93  1.932 2.721 3.375 3.185]
 [4.056 1.135 0.787 4.056 4.902 3.014]
 [0.93  2.376 1.55  0.93  1.208 1.826]
 [3.136 1.898 1.269 3.136 3.83  2.877]
 [0.846 4.844 3.146 0.846 1.206 3.099]
 [2.157 4.917 3.211 2.157 2.778 3.916]
 [1.826 1.825 1.205 1.826 2.258 2.061]
 [1.226 4.198 2.733 1.226 1.635 2.979]]

[[ 1.287  1.081]
 [ 2.274  0.284]
 [ 0.289  0.934]
 [ 1.646  0.639]
 [-0.032  1.951]
 [ 0.735  1.923]
 [ 0.879  0.667]
 [ 0.263  1.669]]

[[1.726 0.188 0.144 1.726 2.074 1.125]
 [0.462 2.486 1.615 0.462 0.652 1.607]]



**prediction**

In [27]:
# new user data
# user has given the 1st, 2nd, 4th, and 6th products the ratings 2, 2, 1, and 2, respectively
#need to predict the remainig ratings
new_user_indices = [1, 2, 4, 5]
new_user_ratings = [2, 2, 1, 2]

In [28]:
new_user_P_row_initial = np.random.rand(1, K)
new_user_P_row = tf.Variable(new_user_P_row_initial, dtype=tf.float32)
new_user_P_row_times_Q = tf.matmul(new_user_P_row, final_Q_matrix)
res = tf.gather(new_user_P_row_times_Q, new_user_indices, axis=1)
squared_error = tf.square(new_user_ratings - res)
loss = tf.reduce_sum(squared_error)
optimizer = tf.train.GradientDescentOptimizer(0.01)
predict = optimizer.minimize(loss)

In [29]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(50000):
    sess.run(predict)

In [30]:
final_new_user_P_row_times_Q = np.around(sess.run(new_user_P_row_times_Q), 3)
print(np.round(final_new_user_P_row_times_Q))

[[1. 2. 2. 1. 1. 2.]]
